<a href="https://colab.research.google.com/github/iberganzo/darknet/blob/master/Hybrid_MSRM_based_DL_and_multitemporal_Sentinel_2_based_ML_algorithm_detects_near_10k_archaeological_tumuli_in_NW_Iberia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hybrid MSRM-based DL and multitemporal Sentinel 2-based ML algorithm to detect archaeological tumuli

Iban Berganzo-Besga1, Hector A. Orengo1*, Felipe Lumbreras2, Miguel Carrero-Pazos3, João Fonte4 , Benito Vilas-Estévez5

1. Landscape Archaeology Research Group, Catalan Institute of Classical Archaeology, Pl. Rovellat s/n, Tarragona, 43003, Spain; iberganzo@icac.cat; horengo@icac.cat
2. Computer vision Center, Computer Science Dept., Universitat Autònoma de Barcelona, Edifici O, Campus UAB, Bellaterra, Barcelona, 08193, Spain; felipe@cvc.uab.es
3. Institute of Archaeology, University College London, 31-34 Gordon Square, LondonWC1H 0PY, UK; miguel.pazos.15@ucl.ac.uk
4. Department of Archaeology, University of Exeter, Laver Building, North Park Road, EX4 4QE, Exeter, UK; j.fonte3@exeter.ac.uk
5. Grupo de Estudos de Arqueoloxía, Antigüidade e Territorio, University of Vigo, Spain; benito.arborearqueoloxia@gmail.com
* Correspondence: horengo@icac.cat

DOI: 
---

# 1. Pre-processing

**1.1. Filtered DTM by GEE**

---

To obtain the filtered DTM (MSRMxRF) of our DTM (resolution of 1 m/px) we must use the provided code (code1.txt) in the GEE Code Editor, Repository and Cloud Computing Platform. GEE requires authentication and allows a maximum file size for our DTM of 1GB. The resulting file will be saved in the associated Google Drive.

**1.2. Image Cropping**

---

In [24]:
from PIL import Image

infile = 'MSRM.tif'
chopsize = 1024
n0 = 0

img = Image.open(infile)
width, height = img.size

for x0 in range(0, width, chopsize):
  for y0 in range(0, height, chopsize):
    box = (x0, y0,
          x0+chopsize if x0+chopsize <  width else  width - 1,
          y0+chopsize if y0+chopsize < height else height - 1)
    # print('%s %s' % (infile, box))
    img.crop(box).save('%s.%05d.tif' % (infile.replace('.tif',''), n0))
    n0 = n0 + 1

**1.3. Image Conversion**

---

The conversion of the TIFF images into PNG has been done using Photoshop. The images were first converted to JPEG (quality: 5) using the image processor and then saved as PNG using an automated action which is provided (JPEGtoPNG.atn).

# 2. Processing

**2.1. Initialization**

---

First of all, edit the notebook configuration to **use GPU**



In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print ("GPU Available: ", tf.test.is_gpu_available())
print (tf.__version__)
from tensorflow import keras
print (tf.keras.__version__)
!/usr/local/cuda/bin/nvcc --version

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Available:  True
2.6.0
2.6.0
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [2]:
!git clone https://github.com/iberganzo/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 14764, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (170/170), done.
remote: Total 14764 (delta 61), reused 0 (delta 0), pack-reused 14583
Receiving objects: 100% (14764/14764), 41.00 MiB | 28.96 MiB/s, done.
Resolving deltas: 100% (9987/9987), done.


In [3]:
cd darknet

/content/darknet


In [4]:
!mkdir backup

In [11]:
cd ..

/content


In [12]:
!wget https://mega.nz/linux/MEGAsync/xUbuntu_18.04/amd64/megacmd-xUbuntu_18.04_amd64.deb
!apt install libc-ares2 libcrypto++6 libmediainfo0v5 libzen0v5
!apt --fix-broken install
!dpkg -i megacmd-xUbuntu_18.04_amd64.deb

--2021-09-20 12:37:45--  https://mega.nz/linux/MEGAsync/xUbuntu_18.04/amd64/megacmd-xUbuntu_18.04_amd64.deb
Resolving mega.nz (mega.nz)... 66.203.127.18, 2a0b:e40:3::18
Connecting to mega.nz (mega.nz)|66.203.127.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7647126 (7.3M) [application/x-deb]
Saving to: ‘megacmd-xUbuntu_18.04_amd64.deb’

megacmd-xUbuntu_18. 100%[===================>]   7.29M  6.06MB/s    in 1.2s    

2021-09-20 12:37:48 (6.06 MB/s) - ‘megacmd-xUbuntu_18.04_amd64.deb’ saved [7647126/7647126]

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcrypto++6 is already the newest version (5.6.4-8).
libzen0v5 is already the newest version (0.4.37-1).
libc-ares2 is already the newest version (1.14.0-1ubuntu0.1).
libmediainfo0v5 is already the newest version (17.12-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Reading package lists... Done
Building dependency tree       


In [16]:
!mega-get https://mega.nz/file/NddlzYbQ#Ea_mT8uKiZmHZYrHr0SMJJ5YEXma1Gwqdy5b3cw1GbE /content/darknet/backup/

Download finished: /content/darknet/backup/yolov3-custom_last.weights
TRANSFERRING ||########################################||(234/234 MB: 100.00 %)  


In [14]:
cd darknet

/content/darknet


In [9]:
!make

mkdir -p ./obj/
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
            

**2.2. Training**

---

The algorithm is already trained, so you can proceed to the next step.



**2.3. Detection**

---

The cropped 1024x1024px PNG images from the filtered DTM must be saved in data/objval. Delete previous files in that folder. Now we proceed to the detection of the mounds in them using the YOLOv3 shape detection algorithm

In [20]:
!./darknet detector test data/obj.data cfg/yolov3-custom-detection.cfg backup/yolov3-custom_last.weights data/objval/MSRM263.115.png -dont_show # Individual detection

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1   1024 x1024 x   1 -> 1024 x1024 x  32 0.604 BF
   1 conv     64       3 x 3/ 2   1024 x1024 x  32 ->  512 x 512 x  64 9.664 BF
   2 conv     32       1 x 1/ 1    512 x 512 x  64 ->  512 x 512 x  32 1.074 BF
   3 conv     64       3 x 3/ 1    512 x 512 x  32 ->  512 x 512 x  64 9.664 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 512 x 512 x  64 0.017 BF
   5 conv    128       3 x 3/ 2    512 x 512 x  64 ->  256 x 256 x 128 9.664 BF
   6 conv     64       1 x 1/ 1    256 x 256 x 128 ->  256 x 256 x  64 1.074 BF
   7 conv    128       3 x 3/ 1    256 x 256 x  64 ->  256 x 256 x 128 9.664 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, ou

In [21]:
!python generate_validation.py

!./darknet detector test data/obj.data cfg/yolov3-custom-detection.cfg backup/yolov3-custom_last.weights -dont_show -ext_output < data/val.txt > result.txt # Multiple detection

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1   1024 x1024 x   1 -> 1024 x1024 x  32 0.604 BF
   1 conv     64       3 x 3/ 2   1024 x1024 x  32 ->  512 x 512 x  64 9.664 BF
   2 conv     32       1 x 1/ 1    512 x 512 x  64 ->  512 x 512 x  32 1.074 BF
   3 conv     64       3 x 3/ 1    512 x 512 x  32 ->  512 x 512 x  64 9.664 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 512 x 512 x  64 0.017 BF
   5 conv    128       3 x 3/ 2    512 x 512 x  64 ->  256 x 256 x 128 9.664 BF
   6 conv     64       1 x 1/ 1    256 x 256 x 128 ->  256 x 256 x  64 1.074 BF
   7 conv    128       3 x 3/ 1    256 x 256 x  64 ->  256 x 256 x 128 9.664 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 256 x 256 x 128 0.008 BF
   9 conv     64       1 x 1/ 1    256 x 256 x 128 ->  256 x 25

**2.4. Point extraction**

---

Finally, we must download the resulting file (result.txt) with the list of detected burial mounds and use MATLAB, used for this article, or another programming language to extract the points with the exact coordinates (script3.txt).
Before using MATLAB, the obtained data (result.txt) must be entered into an excel workbook as indicated in the script3.txt. The resulting workbook and its associated point distribution will have the following information for each mound: index, X coordinate, Y coordinate, size in meters, percentage of similarity, and number of the associated PNG image.